# Analysons des factures d'énergie

In [109]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re


!pip install datetime
from datetime import datetime

In [110]:
#open and read csv file
energy_bills = pd.read_csv("Invoice_20201220.csv", sep='|')
display(energy_bills)

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,19767_GEO (0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,19767_GEO (0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,19767_GEO (0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,19767_GEO (0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


#### 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres => Retirer tout autre caractère de la colonne "SITE_NAME".

In [111]:
#pour actualiser une colonne de dataframe il suffit d'écrire : data['nom_colonne'] = nouvelle_colonne

# RegEx  p1 = '.*' zéro ou plus de n'importe quel caractère 
# RegEx p2 = '?' rend la capture non-greedy = plus petite nb possible de caractère jusqu'à la parenthèse fermante 
energy_bills['SITE_NAME'] = energy_bills['SITE_NAME'].str.extract(r'\((.*?)\)')

display(energy_bills)

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,0632KO,FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,055100168,01/07/2020,30/09/2020,2 230,"294,07"


#### 2) Supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.

In [112]:
#method to delete any column (no need to specify the axis)
del energy_bills['INVOICE_NAME']

display(energy_bills)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,01/07/2020,30/09/2020,2 230,"294,07"


#### 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.

In [113]:
#to calcute the number of days between two dates, we use package datetime
# strptime() = string parse time

def days_between(begin_date, end_date):
    date_format = '%d/%m/%Y'
    begin_date_obj = datetime.strptime(begin_date, date_format)
    end_date_obj = datetime.strptime(end_date, date_format)
    diff = end_date_obj - begin_date_obj
    return diff.days

begin_date = energy_bills['BEGIN']
end_date = energy_bills['END']

# Add column 'PERIOD_IN_DAYS'
# function apply is used to apply a function all over an axis, here the horizontal axis (axis=1)
# lambda = anonymous function in one line for temporary use. Lambda function only got 1 expression (no more)
energy_bills['PERIOD_DAYS'] = energy_bills.apply(lambda row: days_between(row['BEGIN'], row['END']), axis=1)

# to move 'BILLING_PERIOD_IN_DAYS' column
col_billing_period = energy_bills.pop('PERIOD_DAYS')
energy_bills.insert(loc=energy_bills.columns.get_loc('END')+1, column='PERIOD_DAYS', value=col_billing_period)

display(energy_bills)

,SITE_NAME,BEGIN,END,PERIOD_DAYS,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,30,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,27,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,30,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,29,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,30,2 574,"333,37"
...,...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,123,3 028,"397,18"
51428,0632KO,01/01/2020,31/03/2020,90,2 163,"283,57"
51429,0632KO,01/04/2020,30/06/2020,90,2 160,"312,55"
51430,0632KO,01/07/2020,30/09/2020,91,2 230,"294,07"


#### 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.

In [114]:
#to delete space into 'KWH_PERIOD' then convert in integer
energy_bills['KWH_PERIOD'] = energy_bills['KWH_PERIOD'].astype(str).str.replace(' ', '').astype(int)

#to create new column
energy_bills['AV_KWH_DAY'] = energy_bills['KWH_PERIOD'] / energy_bills['PERIOD_DAYS']

#to place this new column in other place into dataframe
energy_bills.insert(5, 'AV_KWH_DAY', energy_bills.pop('AV_KWH_DAY'))

display(energy_bills)

,SITE_NAME,BEGIN,END,PERIOD_DAYS,KWH_PERIOD,AV_KWH_DAY,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,30,2447,81.566667,"316,46"
1,0001BB,01/02/2018,28/02/2018,27,2211,81.888889,"286,82"
2,0001BB,01/03/2018,31/03/2018,30,2450,81.666667,"317,32"
3,0001BB,01/04/2018,30/04/2018,29,2431,83.827586,"315,00"
4,0001BB,01/05/2018,31/05/2018,30,2574,85.800000,"333,37"
...,...,...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,123,3028,24.617886,"397,18"
51428,0632KO,01/01/2020,31/03/2020,90,2163,24.033333,"283,57"
51429,0632KO,01/04/2020,30/06/2020,90,2160,24.000000,"312,55"
51430,0632KO,01/07/2020,30/09/2020,91,2230,24.505495,"294,07"


#### 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

In [115]:
#gathering bills by site
bills = energy_bills['SITE_NAME'].value_counts()

#push my datas into a new dataframe called bills_by_site
bills_by_site = pd.DataFrame({'SITE_NAME': bills.index, 'BILLS': bills.values})

# pd.merge to combine 2 dataframes
# 'on=' tells wich column we'll use as key to merge
# 'how=' means leftjoin and garantees we keep all 'energy_bills' rows (as my dataframe is on left, cause it's my first argument in my function)
energy_bills = pd.merge(energy_bills, bills_by_site, on='SITE_NAME', how='left')

display(energy_bills)

,SITE_NAME,BEGIN,END,PERIOD_DAYS,KWH_PERIOD,AV_KWH_DAY,PERIOD_EUR,BILLS
0,0001BB,01/01/2018,31/01/2018,30,2447,81.566667,"316,46",35
1,0001BB,01/02/2018,28/02/2018,27,2211,81.888889,"286,82",35
2,0001BB,01/03/2018,31/03/2018,30,2450,81.666667,"317,32",35
3,0001BB,01/04/2018,30/04/2018,29,2431,83.827586,"315,00",35
4,0001BB,01/05/2018,31/05/2018,30,2574,85.800000,"333,37",35
...,...,...,...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,123,3028,24.617886,"397,18",4
51428,0632KO,01/01/2020,31/03/2020,90,2163,24.033333,"283,57",4
51429,0632KO,01/04/2020,30/06/2020,90,2160,24.000000,"312,55",4
51430,0632KO,01/07/2020,30/09/2020,91,2230,24.505495,"294,07",4


#### 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.

In [95]:
mean_by_site = (energy_bills.groupby('SITE_NAME')['AV_KWH_DAY'].mean())
std_by_site = (energy_bills.groupby('SITE_NAME')['AV_KWH_DAY'].std())

# Calculate the coef of variation for the cosumption
coef_of_variation = ((std_by_site / mean_by_site) * 100)

#Convert in list
mean_list = mean_by_site.tolist()
std_list = std_by_site.tolist()
coef_of_variation_list = coef_of_variation.tolist()

display(coef_of_variation)

SITE_NAME
           47.084502
0001BB      6.606808
0001BR      5.048534
0001KO     10.447159
0002BB      6.684147
             ...    
9513BB      0.376722
9515BB    117.872141
9516BB      1.812700
9517BB      0.055898
9518BB      0.311913
Name: AV_KWH_DAY, Length: 2403, dtype: float64

#### 7) Enfin, exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi.

In [116]:
def save_list_to_csv(data_list, file_name):
    # Créer un DataFrame à partir de la liste
    df = pd.DataFrame(data_list)
    
    # Enregistrer le DataFrame dans un fichier CSV
    df.to_csv(file_name, index=False)

save_list_to_csv(coef_of_variation_list, 'coef_of_variation_list.csv')

In [117]:
energy_bills.to_csv('Invoice_20201220_cleaned.csv', index=False)

## End!